In [36]:
%load_ext autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
%autoreload 2

In [38]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
import sys
sys.path.append('/home/ext_navish_iitkgp_gmail_com/ganRegression/')

In [41]:
import numpy as np
import os
from functools import partial
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import sys
import pickle
from pathlib import Path


# hex2vec
HOME = os.environ["HOME"]

sys.path.insert(0, f"{HOME}/hex2vec")

# add codebase
sys.path.insert(0, f"/gcsmount-notebook/codebase")

from src.data.make_dataset import h3_to_polygon
import urban_tools.constants as uc
from urban_tools.hex_pipeline import RouteHexHandler, TestTrainManager
from urban_tools.pipelines import route_hex_pipeline


## Read in the Delivery DataFrame
p = Path("/gcsmount-research-data-staging/osmnx-cities/hexed-routes/debug-multi-synthetic-tags/hh.pkl")
hh = RouteHexHandler.from_pickle(p)
# embedding_df = pd.read_pickle("/gcsmount-research-data-staging/osmnx-cities/hexed-complete/Boston, MA/boston_embedding.pkl")

### Remove Super Tags
# hh.drop_super_tags()
### Remove Sub Tags
# hh = hh.drop_sub_tags()
### Filter for only H3 with > X Data Points

hh = hh.filter_hex_occurance(20)
## Drop Unecessary Columns
hh.other_tags
### Drop Chicago
# print(tagged_df.shape[0].compute(), tagged_df.shape[1])
# tagged_df = tagged_df.loc[~tagged_df["city"].str.contains("Chicago")]
# filter for only Boston
hh.df = hh.df.loc[hh.df.city.str.contains("Boston")]
# print(tagged_df.shape[0].compute(), tagged_df.shape[1])

## Data Preprocessing: Scaling and Splitting into train test


# create a grouped dataframe
tagged_df = hh.df.groupby("h3").agg({
    "planned_service_time_log": "mean",
    **{
        tag: "first"
        for tag in hh.all_tags
    }
})


from sklearn.preprocessing import RobustScaler

tt = TestTrainManager(
    tagged_df,
    x_col=hh.all_tags,
    y_col=["planned_service_time_log"],
    scaler=RobustScaler,
    grouped=True,
)
tt.split_test_train(train_size=0.8, random_seed=6781)


tt.scale_test_train()
tt.build_test_df(agg=False)

In [52]:
X_train, y_train, X_test, y_test = tt.X_train.values, tt.Y_train.values.ravel(), tt.X_test.values, tt.Y_test.loc.values

In [71]:
import matplotlib.pyplot as plt # creating visualizations
import numpy as np # basic math and random numbers
import torch # package for building functions with learnable parameters
import torch.nn as nn # prebuilt functions specific to neural mdns
from torch.autograd import Variable # storing data while learning
import torch.nn.functional as F # prebuilt functions for neural mdns

In [97]:
class MDN(nn.Module):
    def __init__(self, n_hidden, n_gaussians):
        super(MDN, self).__init__()
        # hidden layer
        self.z_h = nn.Linear(957, n_hidden)
        # pi layer
        self.z_pi = nn.Linear(n_hidden, n_gaussians)
        # sigma layer 
        self.z_sigma = nn.Linear(n_hidden, n_gaussians) 
        # mu layer
        self.z_mu = nn.Linear(n_hidden, n_gaussians)

    def forward(self, x):
        z_h = F.tanh(self.z_h(x))
        pi = F.softmax(self.z_pi(z_h), -1)
        sigma = torch.exp(self.z_sigma(z_h))
        mu = self.z_mu(z_h)
        return pi, sigma, mu

In [98]:
oneDivSqrtTwoPI = 1.0 / np.sqrt(2.0*np.pi) # normalization factor for Gaussians
def gaussian_distribution(y, mu, sigma):
    # make |mu|=K copies of y, subtract mu, divide by sigma
    result = (y.expand_as(mu) - mu) * torch.reciprocal(sigma)
    result = -0.5 * (result * result)
    return (torch.exp(result) * torch.reciprocal(sigma)) * oneDivSqrtTwoPI

def mdn_loss_fn(pi, sigma, mu, y):
    result = gaussian_distribution(y, mu, sigma) * pi
    result = torch.sum(result, dim=1)
    result = -torch.log(result)
    return torch.mean(result)

In [99]:
mdn = MDN(n_hidden=20, n_gaussians=5)

In [100]:
optimizer = torch.optim.Adam(mdn.parameters())

In [101]:
# change data type and shape, move from numpy to torch
x_tensor = torch.Tensor(X_train)
y_tensor = torch.Tensor(y_train)
x_variable = Variable(x_tensor)
y_variable = Variable(y_tensor, requires_grad=False)

# ------------------------------------------------------------------------------

mdn_x_data = y_train
mdn_y_data = X_train

mdn_x_tensor = y_tensor
mdn_y_tensor = x_tensor

x_variable = Variable(mdn_x_tensor)
y_variable = Variable(mdn_y_tensor, requires_grad=False)

In [102]:
def train_mdn():
    for epoch in range(100):
        pi_variable, sigma_variable, mu_variable = mdn(x_variable)
        loss = mdn_loss_fn(pi_variable, sigma_variable, mu_variable, y_variable)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}: loss = {loss.data[0]}')

train_mdn()

RuntimeError: expand(torch.FloatTensor{[957, 674]}, size=[5]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)